In [1]:
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,SnowballStemmer,WordNetLemmatizer
import json
import os
from tqdm.notebook import tqdm,tnrange
import string
import numpy as np
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from collections import defaultdict

nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /home/sandeep/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sandeep/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Query

In [2]:
class Response():
    def __init__(self,data):
        self.data=set(data)

    def getMapping(self,file):
        self.mapping=json.load(open(file))
        for i in self.data:
            print(self.mapping[i])
    
    def __str__(self):
        return str(len(self.data))

    def add(self,response):
        return Response(set.union(self.data,response.data))
        
    def intersect(self,response):
        return Response(set.intersection(self.data,response.data))

    def diff(self,response):
        return Response(set.difference(self.data,response.data))
        

In [44]:
class Query():
    def __init__(self,file):
        self.db=json.load(open(file))
        self.db=defaultdict(lambda:[],self.db)        

    def OR(self,term1,term2):
        return term1.add(term2)
    
    def AND(self,term1,term2):
        return term1.intersect(term2)
    
    def ANDNOT(self,term1,term2):
        univ=Response(np.arange(468))
        not_term2=univ.diff(term2)
        return term1.intersect(not_term2)

    def ORNOT(self,term1,term2):
        univ=Response(np.arange(468))
        not_term2=univ.diff(term2)
        joint = term1.intersect(term2)
        return not_term2.add(joint)

    def no_comparisonsOR(self,term1, term2):
        first = self.db[term1]
        second =self.db[term2]
        i,j,count=0,0,0
        ans = []
        while(i<len(first) and j<len(second)):
            count+=1
            if(first[i]<second[j]):
                ans.append(first[i])
                i+=1          
            elif(first[i]>second[j]):
                ans.append(second[j])
                j+=1
            else:
                ans.append(first[i])
                i+=1
                j+=1
        
        ans.extend(first[i:])
        ans.extend(second[j:])

        return count    

    def no_comparisonsAND(self,term1, term2):
        first = self.db[term1]
        second =self.db[term2]
        i,j,count=0,0,0
        ans = []
        while(i<len(first) and j<len(second)):
            count+=1
            if(first[i]<second[j]):
                i+=1          
            elif(first[i]>second[j]):
                j+=1
            else:
                ans.append(first[i])
                i+=1
                j+=1
        return count
    
    def no_comparisonsANDNOT(self,term1, term2):
        temp=Response(np.arange(468)) 
        second = Response(self.db[term2]) #list of term2
        second=temp.diff(second) #generating Universal - (Not terms2)
        second=list(second.data)
        second.sort() #sorting, if needed
        first = list(set(self.db[term1])) #list of term1
        first.sort()
        #initiating general AND operator
        i,j,count=0,0,0
        ans = []
        while(i<len(first) and j<len(second)):
            count+=1
            if(first[i]<second[j]):
                i+=1          
            elif(first[i]>second[j]):
                j+=1
            else:
                ans.append(first[i])
                i+=1
                j+=1
        return count
    
    def no_comparisonsORNOT(self,term1, term2):
        temp=Response(np.arange(468)) 
        second = Response(self.db[term2]) #list of term2
        second=temp.diff(second) #generating Universal - (Not terms2)
        second=list(second.data)
        second.sort() #sorting, if needed
        first = self.db[term1] #list of term1
    
        i,j,count=0,0,0
        ans = []
        while(i<len(first) and j<len(second)):
            count+=1
            if(first[i]<second[j]):
                ans.append(first[i])
                i+=1          
            elif(first[i]>second[j]):
                ans.append(second[j])
                j+=1
            else:
                ans.append(first[i])
                i+=1
                j+=1
        ans.extend(first[i:])
        ans.extend(second[j:])
        return count  

    def stripSpecialChar(self,text):
        return ''.join(ch for ch in text if ch.isalnum() and not ch.isdigit() and ch not in string.punctuation)

    def preProcess(self,text):
        stemmer = SnowballStemmer("english")
        stopWords = set(stopwords.words('english'))

        text = text.lower()
        text_tokens = word_tokenize(text)
        stemmedWords = list([stemmer.stem(word) for word in text_tokens])

        validTokens = [i for i in stemmedWords if i not in stopWords]
        validTokens = [self.stripSpecialChar(x) for x in validTokens]
        validTokens = [x for x in validTokens if len(x) > 1]
        return validTokens
    
    def processQuery(self,inp,ops):
        terms=self.preProcess(inp)
        print(terms)
        output=Response(self.db[terms[0]])
        comparisons=0
        for i in tnrange(1,len(terms)):
            curr=Response(self.db[terms[i]])
            if(ops[i-1]=='OR'):
                output=self.OR(curr,output)
                comparisons+=self.no_comparisonsOR(terms[i],terms[i-1])
            elif(ops[i-1]=='AND'):
                output=self.AND(curr,output)
                comparisons+=self.no_comparisonsAND(terms[i],terms[i-1])
            elif(ops[i-1]=='OR NOT'):
                output=self.ORNOT(curr,output)
                comparisons+=self.no_comparisonsORNOT(terms[i],terms[i-1])
            elif(ops[i-1]=='AND NOT'):
                output=self.ANDNOT(curr,output)
                comparisons+=self.no_comparisonsANDNOT(terms[i],terms[i-1])

        print("Number of documents matched:",output)
        print("No. of comparisons required:",comparisons)


In [45]:
inp="lion stood thoughtfully for a moment"
ops=['OR',"OR","OR"]
# inp="telephone, paved, roads"
# ops=['OR NOT',"AND NOT"]

query=Query("output.json")
query.processQuery(inp,ops)


['telephon', 'pave', 'road']


  0%|          | 0/2 [00:00<?, ?it/s]

Number of documents matched: 14
No. of comparisons required: 870
